# Proyecto I 
* Monica Alfaro Parrales
* Adrián Ramírez Mattey
* Gilberth Rodríguez Mejías 

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import RobustScaler
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from IPython.display import display
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_curve, roc_auc_score, confusion_matrix

## Carga del dataset "Pima Indians Diabetes Database"

In [ ]:
data = pd.read_csv('diabetes.csv')
print("data head")
display(data.head())
print("data describe")
display(data.describe())
display(data)

X = data[[
    'Pregnancies',
    'Glucose',
    'BloodPressure',
    'SkinThickness',
    'Insulin',
    'BMI',
    'DiabetesPedigreeFunction',
    'Age'
]].values

y = data['Outcome']

## Análisis de datos
1 = Diabetes

0 = No diabetes

### Gráficos de dispersión 

In [ ]:
sns.scatterplot(x='Pregnancies', y='Age', hue='Outcome', data=data, )

# Mostrar el gráfico
plt.title("Gráfico de Dispersión de Embarazos")
plt.xlabel("Embarazos")
plt.ylabel("Edad")
plt.show()

In [ ]:
sns.scatterplot(x='Glucose', y='Age', hue='Outcome', data=data, )

# Mostrar el gráfico
plt.title("Gráfico de Dispersión de Glucosa")
plt.xlabel("Glucosa")
plt.ylabel("Edad")
plt.show()

In [ ]:
sns.scatterplot(x='BloodPressure', y='Age', hue='Outcome', data=data, )

# Mostrar el gráfico
plt.title("Gráfico de Dispersión de Presión Arterial")
plt.xlabel("Presión arterial")
plt.ylabel("Edad")
plt.show()

In [ ]:
sns.scatterplot(x='SkinThickness', y='Age', hue='Outcome', data=data, )

# Mostrar el gráfico
plt.title("Gráfico de Dispersión de Grosor de la piel")
plt.xlabel("Grosor de la piel")
plt.ylabel("Edad")
plt.show()

In [ ]:
sns.scatterplot(x='Insulin', y='Age', hue='Outcome', data=data, )

# Mostrar el gráfico
plt.title("Gráfico de Dispersión de Insulina")
plt.xlabel("Insulina")
plt.ylabel("Edad")
plt.show()

In [ ]:
sns.scatterplot(x='BMI', y='Age', hue='Outcome', data=data, )

# Mostrar el gráfico
plt.title("Gráfico de Dispersión de BMI")
plt.xlabel("BMI")
plt.ylabel("Edad")
plt.show()

In [ ]:
sns.scatterplot(x='DiabetesPedigreeFunction', y='Age', hue='Outcome', data=data, )

# Mostrar el gráfico
plt.title("Gráfico de Dispersión de Diabetes Pedigree")
plt.xlabel("Diabetes Pedigree")
plt.ylabel("Edad")
plt.show()


### Histogramas

In [ ]:
# Contar la cantidad de datos con Outcome 1 y 0
outcome_counts = data['Outcome'].value_counts()

# Crear el histograma
plt.bar(outcome_counts.index, outcome_counts.values, color=['blue', 'green'])
plt.xlabel('Outcome')
plt.ylabel('Cantidad de datos')
plt.title('Cantidad de datos por tipo de Outcome')
plt.xticks([0, 1], ['Outcome 0', 'Outcome 1'])
plt.show()

In [ ]:
dataGroup = data.groupby('Outcome')
plt.figure(figsize=(15, 8))
plt.hist(dataGroup['Pregnancies'].get_group(0), bins=30, alpha=0.5, label='0')
plt.hist(dataGroup['Pregnancies'].get_group(1), bins=30, alpha=0.5, label='1')
plt.title('Histograma de Diabetes - Embarazos')
plt.xlabel('Embarazos')
plt.ylabel('Personas')
plt.grid(True)
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(15, 8))
plt.hist(dataGroup['Glucose'].get_group(0), bins=30, alpha=0.5, label='0')
plt.hist(dataGroup['Glucose'].get_group(1), bins=30, alpha=0.5, label='1')
plt.title('Histograma de Diabetes - Glucosa')
plt.xlabel('Glucosa')
plt.ylabel('Personas')
plt.grid(True)
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(15, 8))
plt.hist(dataGroup['Insulin'].get_group(0), bins=30, alpha=0.5, label='0')
plt.hist(dataGroup['Insulin'].get_group(1), bins=30, alpha=0.5, label='1')
plt.title('Histograma de Diabetes - Insulina')
plt.xlabel('Insulina')
plt.ylabel('Personas')
plt.grid(True)
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(15, 8))
plt.hist(dataGroup['BMI'].get_group(0), bins=30, alpha=0.5, label='0')
plt.hist(dataGroup['BMI'].get_group(1), bins=30, alpha=0.5, label='1')
plt.title('Histograma de Diabetes - BMI')
plt.xlabel('BMI')
plt.ylabel('Personas')
plt.grid(True)
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(15, 8))
plt.hist(dataGroup['DiabetesPedigreeFunction'].get_group(0), bins=30, alpha=0.5, label='0')
plt.hist(dataGroup['DiabetesPedigreeFunction'].get_group(1), bins=30, alpha=0.5, label='1')
plt.title('Histograma de Diabetes - DiabetesPedigreeFunction')
plt.xlabel('DiabetesPedigreeFunction')
plt.ylabel('Personas')
plt.grid(True)
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(15, 8))
plt.hist(dataGroup['Age'].get_group(0), bins=30, alpha=0.5, label='0')
plt.hist(dataGroup['Age'].get_group(1), bins=30, alpha=0.5, label='1')
plt.title('Histograma de Diabetes - Edad')
plt.xlabel('Edad')
plt.ylabel('Personas')
plt.grid(True)
plt.legend()
plt.show()

## Eliminación de outliers y división del dataset
Se procede a hacer uso de IQR, el rango intercuartil (IQR) es la diferencia entre el percentil 75 y el 25 de los datos. Es una medida de dispersión similar a la desviación estándar o la varianza, pero es mucho más robusta frente a valores atípicos. Posteriormente, se hace un shuffle de la data para asegurar una mayor distribución y se divide el dataset en un 80% training y 20% testing. 

In [ ]:
# Calcula el IQR
Q1 = data.quantile(0.25)
Q3 = data.quantile(0.75)
IQR = Q3 - Q1

# Identifica los outliers
outliers = ((data < (Q1 - 1.5 * IQR)) | (data > (Q3 + 1.5 * IQR)))

# Encuentra las filas que tienen al menos un outlier
outliers_indices = outliers.any(axis=1)

# Muestra las filas con outliers
print("Filas con outliers:")
display(data[outliers_indices])

# Elimina los outliers
data_sin_outliers = data[~outliers_indices]

# Muestra el conjunto de datos sin outliers
print("Data sin outliers:")
display(data_sin_outliers)

# Estadísticas del conjunto de datos sin outliers
print("Data sin outliers")
display(data_sin_outliers.describe())

### Gráficos de dispersión posterior a eliminación de outliers

In [ ]:
sns.scatterplot(x='Pregnancies', y='Age', hue='Outcome', data=data_sin_outliers, )

# Mostrar el gráfico
plt.title("Gráfico de Dispersión de Embarazos (no outliers)")
plt.xlabel("Embarazos")
plt.ylabel("Edad")
plt.show()

In [ ]:
sns.scatterplot(x='Glucose', y='Age', hue='Outcome', data=data_sin_outliers )

# Mostrar el gráfico
plt.title("Gráfico de Dispersión de Glucosa (no outliers)")
plt.xlabel("Glucosa")
plt.ylabel("Edad")
plt.show()


In [ ]:
sns.scatterplot(x='BloodPressure', y='Age', hue='Outcome', data=data_sin_outliers, )

# Mostrar el gráfico
plt.title("Gráfico de Dispersión de Presión Arterial (no outliers)")
plt.xlabel("Presión arterial")
plt.ylabel("Edad")
plt.show()


In [ ]:
sns.scatterplot(x='SkinThickness', y='Age', hue='Outcome', data=data_sin_outliers, )

# Mostrar el gráfico
plt.title("Gráfico de Dispersión de Grosor de la piel (no outliers)")
plt.xlabel("Grosor de la piel")
plt.ylabel("Edad")
plt.show()

In [ ]:
sns.scatterplot(x='Insulin', y='Age', hue='Outcome', data=data_sin_outliers )

# Mostrar el gráfico
plt.title("Gráfico de Dispersión de Insulina (no outliers)")
plt.xlabel("Insulina")
plt.ylabel("Edad")
plt.show()

In [ ]:
sns.scatterplot(x='BMI', y='Age', hue='Outcome', data=data_sin_outliers)

# Mostrar el gráfico
plt.title("Gráfico de Dispersión de BMI (no outliers)")
plt.xlabel("BMI")
plt.ylabel("Edad")
plt.show()

In [ ]:
sns.scatterplot(x='DiabetesPedigreeFunction', y='Age', hue='Outcome', data=data_sin_outliers, )

# Mostrar el gráfico
plt.title("Gráfico de Dispersión de Diabetes Pedigree (no outliers)")
plt.xlabel("Diabetes Pedigree")
plt.ylabel("Edad")
plt.show()

### Actualizacion de datos en 0 y normalización

In [ ]:
'''
Media de los datos sin outliers
Pregnancies	Glucose	    BloodPressure	SkinThickness	Insulin	    BMI	        DiabetesPedigreeFunction	Age
3.804382	119.112676	72.120501	    20.563380	    65.931142	32.00579	0.429177	                32.715180
'''
data_sin_ceros = data_sin_outliers.copy()
data_sin_ceros.loc[data_sin_ceros['Pregnancies'] == 0, 'Pregnancies'] = 4 #Numeros enteros en el data set se rendondea 3.804382 a 4
data_sin_ceros.loc[data_sin_ceros['Glucose'] == 0, 'Glucose'] = 120 #Numeros enteros en el data set se rendondea  119.112676 a 120
data_sin_ceros.loc[data_sin_ceros['BloodPressure'] == 0, 'BloodPressure'] = 72 #Numeros enteros en el data set se rendondea 72.120501 a 72
data_sin_ceros.loc[data_sin_ceros['SkinThickness'] == 0, 'SkinThickness'] = 21 #Numeros enteros en el data set se rendondea 20.563380 a 21
data_sin_ceros.loc[data_sin_ceros['Insulin'] == 0, 'Insulin'] = 66 #Numeros enteros en el data set se rendondea 65.931142 a 80
data_sin_ceros.loc[data_sin_ceros['BMI'] == 0, 'BMI'] = 32.0 #Numeros enteros en el data set se rendondea 32.00579 a 32.0
data_sin_ceros.loc[data_sin_ceros['DiabetesPedigreeFunction'] == 0, 'DiabetesPedigreeFunction'] = 0.429 #Numeros enteros en el data set se rendondea 0.429177 a 0.429
print("Data sin ceros")
display(data_sin_ceros.head())

'''
Normalizacion de los datos
Minimos
Pregnancies	Glucose	    BloodPressure	SkinThickness	Insulin	    BMI	        DiabetesPedigreeFunction	Age
0.000000	44.000000	38.000000	    0.000000	    0.000000	18.20000	0.078000	                21.000000

Maximos
Pregnancies	Glucose	    BloodPressure	SkinThickness	Insulin	    BMI	        DiabetesPedigreeFunction	Age
13.000000	198.000000	106.000000	    60.000000	    318.000000	50.00000	1.191000	                66.000000	
'''
data_normalizada = data_sin_ceros.copy()
data_normalizada['Pregnancies'] = (data_normalizada['Pregnancies'] - data_normalizada['Pregnancies'].min()) / (data_normalizada['Pregnancies'].max() - data_normalizada['Pregnancies'].min())
data_normalizada['Glucose'] = (data_normalizada['Glucose'] - data_normalizada['Glucose'].min()) / (data_normalizada['Glucose'].max() - data_normalizada['Glucose'].min())
data_normalizada['BloodPressure'] = (data_normalizada['BloodPressure'] - data_normalizada['BloodPressure'].min()) / (data_normalizada['BloodPressure'].max() - data_normalizada['BloodPressure'].min())
data_normalizada['SkinThickness'] = (data_normalizada['SkinThickness'] - data_normalizada['SkinThickness'].min()) / (data_normalizada['SkinThickness'].max() - data_normalizada['SkinThickness'].min())
data_normalizada['Insulin'] = (data_normalizada['Insulin'] - data_normalizada['Insulin'].min()) / (data_normalizada['Insulin'].max() - data_normalizada['Insulin'].min())
data_normalizada['BMI'] = (data_normalizada['BMI'] - data_normalizada['BMI'].min()) / (data_normalizada['BMI'].max() - data_normalizada['BMI'].min())
data_normalizada['DiabetesPedigreeFunction'] = (data_normalizada['DiabetesPedigreeFunction'] - data_normalizada['DiabetesPedigreeFunction'].min()) / (data_normalizada['DiabetesPedigreeFunction'].max() - data_normalizada['DiabetesPedigreeFunction'].min())
data_normalizada['Age'] = (data_normalizada['Age'] - data_normalizada['Age'].min()) / (data_normalizada['Age'].max() - data_normalizada['Age'].min())
print("Data normalizada")
display(data_normalizada.head())


### División del dataset

In [ ]:
# Antes de dividir los datos, barajamos el dataset de forma aleatoria para mayor diversidad de datos
dataset_shuffled = shuffle(data_sin_outliers, random_state=20)  # random_state para reproducibilidad
dataset_shuf_sin_ceros = shuffle(data_sin_ceros, random_state=20)  # random_state para reproducibilidad
dataset_shuf_normalizada = shuffle(data_normalizada, random_state=20)  # random_state para reproducibilidad
# random_state = 50 malos resultados en el modelo || random_state = 20 buenos resultados en el modelo

# Dividir los datos en conjunto de entrenamiento y prueba
p_train = 0.8  # Porcentaje de training set
train_index = int(len(dataset_shuffled) * p_train)

dataFrameTraining = dataset_shuffled[:train_index]
dataFrameTraining_sin_ceros = dataset_shuf_sin_ceros[:train_index]
dataFrameTraining_normalizada = dataset_shuf_normalizada[:train_index]

dataFrameTesting = dataset_shuffled[train_index:]
dataFrameTesting_sin_ceros = dataset_shuf_sin_ceros[train_index:]
dataFrameTesting_normalizada = dataset_shuf_normalizada[train_index:]

print("Ejemplos usados para entrenar: ", len(dataFrameTraining))
print("Ejemplos usados para test: ", len(dataFrameTesting))

### Funcion para dividir los datos en x_train, y_train, x_test, y_test

In [ ]:
#
def xyDivide(dataFrameTraining, dataFrameTesting):
    x_train = dataFrameTraining[[
    'Pregnancies',
    'Glucose',
    'BloodPressure',
    'SkinThickness',
    'Insulin',
    'BMI',
    'DiabetesPedigreeFunction',
    'Age'
    ]].values

    y_train = dataFrameTraining['Outcome']

    x_test = dataFrameTesting[[
        'Pregnancies',
        'Glucose',
        'BloodPressure',
        'SkinThickness',
        'Insulin',
        'BMI',
        'DiabetesPedigreeFunction',
        'Age'
    ]].values

    y_test = dataFrameTesting['Outcome']
    return x_train, y_train, x_test, y_test

### Funciones para graficar las metricas

In [ ]:
def printAccGraph(lst):
    acc = []
    for i in range(0, len(lst)):
        acc.append(lst[i]['acc'])
    plt.plot(acc)
    plt.title('Accuracy')
    plt.xlabel('Iteraciones')
    plt.ylabel('Accuracy')
    plt.grid(True)
    plt.show()

def printPrecGraph(lst):
    prec = []
    for i in range(0, len(lst)):
        prec.append(lst[i]['prec'])
    plt.plot(prec)
    plt.title('Precision')
    plt.xlabel('Iteraciones')
    plt.ylabel('Precision')
    plt.grid(True)
    plt.show()

def printRecGraph(lst):
    rec = []
    for i in range(0, len(lst)):
        rec.append(lst[i]['rec'])
    plt.plot(rec)
    plt.title('Recall')
    plt.xlabel('Iteraciones')
    plt.ylabel('Recall')
    plt.grid(True)
    plt.show()

def printF1Graph(lst):
    f1 = []
    for i in range(0, len(lst)):
        f1.append(lst[i]['f1'])
    plt.plot(f1)
    plt.title('F1')
    plt.xlabel('Iteraciones')
    plt.ylabel('F1')
    plt.grid(True)
    plt.show()

def printAucGraph(lst):
    auc = []
    for i in range(0, len(lst)):
        auc.append(lst[i]['auc'])
    plt.plot(auc)
    plt.title('AUC')
    plt.xlabel('Iteraciones')
    plt.ylabel('AUC')
    plt.grid(True)
    plt.show()

def printROC(roc, k):
    plt.plot(roc[0], roc[1], label=f"K = {k}")
    plt.xlabel("False Positive Rate")
    plt.ylabel("True Positive Rate")
    plt.title(f"ROC Curve k = {k}")
    plt.grid(True)
    plt.legend()
    plt.show()

def printConfusionMatrix(cm, k):
    plt.figure(figsize=(8, 8))
    plt.imshow(cm, cmap='bwr')
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            plt.text(j, i, str(cm[i, j]), ha='center', va='center', color='white', fontsize=20)
    plt.xticks([0,1], ['No diabetico', 'Diabetico'])
    plt.yticks([0,1], ['No diabetico', 'Diabetico'])
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.title(f"Matriz Confusión k = {k}")
    plt.colorbar()
    plt.show()

## KNN y metricas del KNN

### KNN con data set sin outliers

In [ ]:
x_train, y_train, x_test, y_test = xyDivide(dataFrameTraining, dataFrameTesting)
lst = []
    
for i in range(1,101):
    knn = KNeighborsClassifier(n_neighbors=i)
    knn.fit(x_train, y_train)
    y_pred = knn.predict(x_test)
    
    acc = accuracy_score(y_test, y_pred)
    prec = precision_score(y_test, y_pred)
    rec = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    roc = roc_curve(y_test, y_pred)
    auc = roc_auc_score(y_test, y_pred)
    matrizConfusion = confusion_matrix(y_test, y_pred)
    lst.append({"k": i, "acc": acc, "prec": prec, "rec": rec, "f1": f1, "auc": auc, "roc": roc, "matrizConfusion": matrizConfusion})


maxAuc = max(lst, key=lambda x:x['auc'])
maxPrec = max(lst, key=lambda x:x['prec'])
maxRec = max(lst, key=lambda x:x['rec'])
maxF1 = max(lst, key=lambda x:x['f1'])
maxAcc = max(lst, key=lambda x:x['acc'])

print("Max Accuracy")
print(f"k = {maxAcc['k']} || AUC = {maxAcc['auc']} || Precision = {maxAcc['prec']} || Recall = {maxAcc['rec']} || F1 = {maxAcc['f1']} || Accuracy = {maxAcc['acc']}")
print("Max Precision")
print(f"k = {maxPrec['k']} || AUC = {maxPrec['auc']} || Precision = {maxPrec['prec']} || Recall = {maxPrec['rec']} || F1 = {maxPrec['f1']} || Accuracy = {maxPrec['acc']}")
print("Max Recall")
print(f"k = {maxRec['k']} || AUC = {maxRec['auc']} || Precision = {maxRec['prec']} || Recall = {maxRec['rec']} || F1 = {maxRec['f1']} || Accuracy = {maxRec['acc']}")
print("Max F1")
print(f"k = {maxF1['k']} || AUC = {maxF1['auc']} || Precision = {maxF1['prec']} || Recall = {maxF1['rec']} || F1 = {maxF1['f1']} || Accuracy = {maxF1['acc']}")
print("Max AUC")
print(f"k = {maxAuc['k']} || AUC = {maxAuc['auc']} || Precision = {maxAuc['prec']} || Recall = {maxAuc['rec']} || F1 = {maxAuc['f1']} || Accuracy = {maxAuc['acc']}")

printAccGraph(lst)
printPrecGraph(lst)
printRecGraph(lst)
printF1Graph(lst)
printAucGraph(lst)

printROC(maxAcc['roc'], maxAcc['k'])
printROC(maxPrec['roc'], maxPrec['k'])
printROC(maxRec['roc'], maxRec['k'])
printROC(maxF1['roc'], maxF1['k'])
printROC(maxAuc['roc'], maxAuc['k'])

printConfusionMatrix(maxAcc['matrizConfusion'], maxAcc['k'])
printConfusionMatrix(maxPrec['matrizConfusion'], maxPrec['k'])
printConfusionMatrix(maxRec['matrizConfusion'], maxRec['k'])
printConfusionMatrix(maxF1['matrizConfusion'], maxF1['k'])
printConfusionMatrix(maxAuc['matrizConfusion'], maxAuc['k'])




### KNN data set sin ceros

In [ ]:
x_train, y_train, x_test, y_test = xyDivide(dataFrameTraining_sin_ceros, dataFrameTesting_sin_ceros)
lst = []

for i in range(1,101):
    knn = KNeighborsClassifier(n_neighbors=i)
    knn.fit(x_train, y_train)
    y_pred = knn.predict(x_test)
    
    acc = accuracy_score(y_test, y_pred)
    prec = precision_score(y_test, y_pred)
    rec = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    roc = roc_curve(y_test, y_pred)
    matrizConfusion = confusion_matrix(y_test, y_pred)
    lst.append({"k": i, "acc": acc, "prec": prec, "rec": rec, "f1": f1, "auc": auc, "roc": roc, "matrizConfusion": matrizConfusion})


maxAuc = max(lst, key=lambda x:x['auc'])
maxPrec = max(lst, key=lambda x:x['prec'])
maxRec = max(lst, key=lambda x:x['rec'])
maxF1 = max(lst, key=lambda x:x['f1'])
maxAcc = max(lst, key=lambda x:x['acc'])

print("Max Accuracy")
print(f"k = {maxAcc['k']} || AUC = {maxAcc['auc']} || Precision = {maxAcc['prec']} || Recall = {maxAcc['rec']} || F1 = {maxAcc['f1']} || Accuracy = {maxAcc['acc']}")
print("Max Precision")
print(f"k = {maxPrec['k']} || AUC = {maxPrec['auc']} || Precision = {maxPrec['prec']} || Recall = {maxPrec['rec']} || F1 = {maxPrec['f1']} || Accuracy = {maxPrec['acc']}")
print("Max Recall")
print(f"k = {maxRec['k']} || AUC = {maxRec['auc']} || Precision = {maxRec['prec']} || Recall = {maxRec['rec']} || F1 = {maxRec['f1']} || Accuracy = {maxRec['acc']}")
print("Max F1")
print(f"k = {maxF1['k']} || AUC = {maxF1['auc']} || Precision = {maxF1['prec']} || Recall = {maxF1['rec']} || F1 = {maxF1['f1']} || Accuracy = {maxF1['acc']}")
print("Max AUC")
print(f"k = {maxAuc['k']} || AUC = {maxAuc['auc']} || Precision = {maxAuc['prec']} || Recall = {maxAuc['rec']} || F1 = {maxAuc['f1']} || Accuracy = {maxAuc['acc']}")

printAccGraph(lst)
printPrecGraph(lst)
printRecGraph(lst)
printF1Graph(lst)
printAucGraph(lst)

printROC(maxAcc['roc'], maxAcc['k'])
printROC(maxPrec['roc'], maxPrec['k'])
printROC(maxRec['roc'], maxRec['k'])
printROC(maxF1['roc'], maxF1['k'])
printROC(maxAuc['roc'], maxAuc['k'])

printConfusionMatrix(maxAcc['matrizConfusion'], maxAcc['k'])
printConfusionMatrix(maxPrec['matrizConfusion'], maxPrec['k'])
printConfusionMatrix(maxRec['matrizConfusion'], maxRec['k'])
printConfusionMatrix(maxF1['matrizConfusion'], maxF1['k'])
printConfusionMatrix(maxAuc['matrizConfusion'], maxAuc['k'])

### KNN data set normalizado

In [ ]:
x_train, y_train, x_test, y_test = xyDivide(dataFrameTraining_normalizada, dataFrameTesting_normalizada)
lst = []

for i in range(1,101):
    knn = KNeighborsClassifier(n_neighbors=i)
    knn.fit(x_train, y_train)
    y_pred = knn.predict(x_test)
    
    acc = accuracy_score(y_test, y_pred)
    prec = precision_score(y_test, y_pred)
    rec = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    roc = roc_curve(y_test, y_pred)
    matrizConfusion = confusion_matrix(y_test, y_pred)
    lst.append({"k": i, "acc": acc, "prec": prec, "rec": rec, "f1": f1, "auc": auc, "roc": roc, "matrizConfusion": matrizConfusion})


maxAuc = max(lst, key=lambda x:x['auc'])
maxPrec = max(lst, key=lambda x:x['prec'])
maxRec = max(lst, key=lambda x:x['rec'])
maxF1 = max(lst, key=lambda x:x['f1'])
maxAcc = max(lst, key=lambda x:x['acc'])

print("Max Accuracy")
print(f"k = {maxAcc['k']} || AUC = {maxAcc['auc']} || Precision = {maxAcc['prec']} || Recall = {maxAcc['rec']} || F1 = {maxAcc['f1']} || Accuracy = {maxAcc['acc']}")
print("Max Precision")
print(f"k = {maxPrec['k']} || AUC = {maxPrec['auc']} || Precision = {maxPrec['prec']} || Recall = {maxPrec['rec']} || F1 = {maxPrec['f1']} || Accuracy = {maxPrec['acc']}")
print("Max Recall")
print(f"k = {maxRec['k']} || AUC = {maxRec['auc']} || Precision = {maxRec['prec']} || Recall = {maxRec['rec']} || F1 = {maxRec['f1']} || Accuracy = {maxRec['acc']}")
print("Max F1")
print(f"k = {maxF1['k']} || AUC = {maxF1['auc']} || Precision = {maxF1['prec']} || Recall = {maxF1['rec']} || F1 = {maxF1['f1']} || Accuracy = {maxF1['acc']}")
print("Max AUC")
print(f"k = {maxAuc['k']} || AUC = {maxAuc['auc']} || Precision = {maxAuc['prec']} || Recall = {maxAuc['rec']} || F1 = {maxAuc['f1']} || Accuracy = {maxAuc['acc']}")

printAccGraph(lst)
printPrecGraph(lst)
printRecGraph(lst)
printF1Graph(lst)
printAucGraph(lst)

printROC(maxAcc['roc'], maxAcc['k'])
printROC(maxPrec['roc'], maxPrec['k'])
printROC(maxRec['roc'], maxRec['k'])
printROC(maxF1['roc'], maxF1['k'])
printROC(maxAuc['roc'], maxAuc['k'])

printConfusionMatrix(maxAcc['matrizConfusion'], maxAcc['k'])
printConfusionMatrix(maxPrec['matrizConfusion'], maxPrec['k'])
printConfusionMatrix(maxRec['matrizConfusion'], maxRec['k'])
printConfusionMatrix(maxF1['matrizConfusion'], maxF1['k'])
printConfusionMatrix(maxAuc['matrizConfusion'], maxAuc['k'])

# Carga del dataset "The Spotify Hit Predictor Dataset (1960-2019)" específico del 2010 al 2019

In [ ]:
data_spotify = pd.read_csv('dataset-of-10s.csv')
print("data head")
display(data_spotify.head())
print("data info")
display(data_spotify.describe())

## Análisis de los datos

Target: Según si apareció en el TOP 100 de Billboard (al menos una vez)

1 = HIT

0 = FLOP

Danceability: "Vibe" de bailabilidad, 0 a 1

Energy: Intensidad de la canción, 0 a 1

Key: llave/nota (-1 cuando no encuentra), positivos y 0

Loudness: Qué tan ruidosa/alta es la canción, valores en dB -60 a 0

Mode: Modalidad del track, escala de 1 (mayor) o 0 (menor) (discreto)

Speechiness: Qué tanta lírica en el track, 0 a 1

*    < 0.33: canciones melódicas, instrumentales
*    \> 0.33 y < 0.66: canciones normales con letra
*    \> 0.66: canciones de solo letra, podcasts, etc...

Accousticness: Medida de confianza sobre si la canción es acústica, de 0 a 1

Instrumentalness: Predice si la canción no tiene voces, de 0 a 1

Liveness: Si hay audiencia en el track, 0 a 1

Valence: Medida que describe el positivismo de la canción, 0 a 1

Tempo: Tempo del track en bits/min, bits/min puede ir de los 10, 200...

Duration in ms: Duración del track en milisegundos

Time Signature: Métrica musical (no entendí 🤯😵)

Chorus Hit: Mejor estimado de cuando el autor inicia el coro, es un timestamp de la canción

Sections: Número de secciones que una canción tiene

### Gráficos de disperción

In [ ]:
# Para ver mejor y label

sns.scatterplot(x='danceability', y='duration_ms', hue='target', data=data_spotify, )
# Mostrar el gráfico
plt.title("Gráfico de Dispersión de Bailabilidad")
plt.xlabel("Bailabilidad")
plt.ylabel("Duración de la canción en ms")
plt.show()

sns.scatterplot(x='danceability', y='sections', hue='target', data=data_spotify, )
# Mostrar el gráfico
plt.title("Gráfico de Dispersión de Bailabilidad")
plt.xlabel("Bailabilidad")
plt.ylabel("Número de secciones")
plt.show()

In [ ]:
sns.scatterplot(x='energy', y='duration_ms', hue='target', data=data_spotify, )
plt.title("Gráfico de Dispersión de Energía")
plt.xlabel("Energía")
plt.ylabel("Duración de la canción en ms")
plt.show()

sns.scatterplot(x='key', y='duration_ms', hue='target', data=data_spotify, )
plt.title("Gráfico de Dispersión de Llave (Nota)")
plt.xlabel("Llave (notación numérica)")
plt.ylabel("Duración de la canción en ms")
plt.show()

sns.scatterplot(x='loudness', y='duration_ms', hue='target', data=data_spotify, )
plt.title("Gráfico de Dispersión de Ruidosidad")
plt.xlabel("Nivel de Ruido")
plt.ylabel("Duración de la canción en ms")
plt.show()

sns.scatterplot(x='mode', y='duration_ms', hue='target', data=data_spotify, )
plt.title("Gráfico de Dispersión de Modalidad (Mayor y Menor)")
plt.xlabel("Modalidad (notación numérica)")
plt.ylabel("Duración de la canción en ms")
plt.show()

sns.scatterplot(x='speechiness', y='duration_ms', hue='target', data=data_spotify, )
plt.title("Gráfico de Dispersión de Nivel de Lírica")
plt.xlabel("Índice de letra en la canción")
plt.ylabel("Duración de la canción en ms")
plt.show()

sns.scatterplot(x='acousticness', y='duration_ms', hue='target', data=data_spotify, )
plt.title("Gráfico de Dispersión de Acústica")
plt.xlabel("Índice de acusticidad")
plt.ylabel("Duración de la canción en ms")
plt.show()

sns.scatterplot(x='instrumentalness', y='duration_ms', hue='target', data=data_spotify, )
plt.title("Gráfico de Dispersión de Instrumentalidad")
plt.xlabel("Índice de instrumentalidad")
plt.ylabel("Duración de la canción en ms")
plt.show()

sns.scatterplot(x='liveness', y='duration_ms', hue='target', data=data_spotify, )
plt.title("Gráfico de Dispersión de Canciones en vivo")
plt.xlabel("Confiabilidad de canción en vivo")
plt.ylabel("Duración de la canción en ms")
plt.show()

sns.scatterplot(x='valence', y='duration_ms', hue='target', data=data_spotify, )
plt.title("Gráfico de Dispersión de Valencia")
plt.xlabel("Nivel de valencia")
plt.ylabel("Duración de la canción en ms")
plt.show()

sns.scatterplot(x='tempo', y='duration_ms', hue='target', data=data_spotify, )
plt.title("Gráfico de Dispersión de Tempo")
plt.xlabel("Tempo de la canción")
plt.ylabel("Duración de la canción en ms")
plt.show()

sns.scatterplot(x='time_signature', y='duration_ms', hue='target', data=data_spotify, )
plt.title("Gráfico de Dispersión de Firma de tiempo")
plt.xlabel("Firma de tiempo de la canción")
plt.ylabel("Duración de la canción en ms")
plt.show()

sns.scatterplot(x='chorus_hit', y='duration_ms', hue='target', data=data_spotify, )
plt.title("Gráfico de Dispersión de Entrada al coro")
plt.xlabel("Tiempo al entrar al coro")
plt.ylabel("Duración de la canción en ms")
plt.show()

sns.scatterplot(x='sections', y='duration_ms', hue='target', data=data_spotify, )
plt.title("Gráfico de Dispersión de Secciones")
plt.xlabel("Cantidad de secciones")
plt.ylabel("Duración de la canción en ms")
plt.show()


### Histograma de balance de las clases

In [ ]:
# Contar la cantidad de datos con Outcome 1 y 0
outcome_counts = data_spotify['target'].value_counts()

# Crear el histograma
plt.bar(outcome_counts.index, outcome_counts.values, color=['blue', 'green'])
plt.xlabel('Clase')
plt.ylabel('Cantidad de datos')
plt.title('Cantidad de datos por tipo de Canción')
plt.xticks([0, 1], ['Flop', 'Hit'])
plt.show()

### Preprocesamiento & Eliminación de Outliers

In [ ]:
# Vamos a intentar usar la versión de Sci-Kit

data_spot_sin_strings = data_spotify.drop(['track', 'artist', 'uri'], axis=1)

transformer = RobustScaler().fit(data_spot_sin_strings)
data_spotify_sin_outliers = transformer.transform(data_spot_sin_strings)

df_spotify_sin_outliers = pd.DataFrame(data_spotify_sin_outliers, columns=data_spot_sin_strings.columns)

display(df_spotify_sin_outliers.head())
display(df_spotify_sin_outliers.describe())

In [ ]:
# Calcula el IQR

data_spotify_sin_strings = data_spotify.drop(['track', 'artist', 'uri'], axis=1)

Q1 = data_spotify_sin_strings.quantile(0.25)
Q3 = data_spotify_sin_strings.quantile(0.75)
IQR = Q3 - Q1

# Identifica los outliers
outliers = ((data_spotify_sin_strings < (Q1 - 1.5 * IQR)) | (data_spotify_sin_strings > (Q3 + 1.5 * IQR)))

# Encuentra las filas que tienen al menos un outlier
outliers_indices = outliers.any(axis=1)

# Muestra las filas con outliers
print("Filas con outliers:")
display(data_spotify_sin_strings[outliers_indices])

# Elimina los outliers
data_spotify_sin_outliers = data_spotify_sin_strings[~outliers_indices]

# Muestra el conjunto de datos sin outliers
print("Data sin outliers:")
display(data_spotify_sin_outliers.head())

# Estadísticas del conjunto de datos sin outliers
print("Descripción Data sin outliers")
display(data_spotify_sin_outliers.describe())

In [ ]:
sns.scatterplot(x='danceability', y='duration_ms', hue='target', data=data_spotify_sin_outliers, )
plt.title("Gráfico de Dispersión de Bailabilidad")
plt.xlabel("Bailabilidad")
plt.ylabel("Duración de la canción en ms")
plt.show()

sns.scatterplot(x='energy', y='duration_ms', hue='target', data=data_spotify_sin_outliers, )
plt.title("Gráfico de Dispersión de Energía")
plt.xlabel("Energía")
plt.ylabel("Duración de la canción en ms")
plt.show()

sns.scatterplot(x='key', y='duration_ms', hue='target', data=data_spotify_sin_outliers, )
plt.title("Gráfico de Dispersión de Llave (Nota)")
plt.xlabel("Llave (notación numérica)")
plt.ylabel("Duración de la canción en ms")
plt.show()

sns.scatterplot(x='loudness', y='duration_ms', hue='target', data=data_spotify_sin_outliers, )
plt.title("Gráfico de Dispersión de Ruidosidad")
plt.xlabel("Nivel de Ruido")
plt.ylabel("Duración de la canción en ms")
plt.show()

sns.scatterplot(x='mode', y='duration_ms', hue='target', data=data_spotify_sin_outliers, )
plt.title("Gráfico de Dispersión de Modalidad (Mayor y Menor)")
plt.xlabel("Modalidad (notación numérica)")
plt.ylabel("Duración de la canción en ms")
plt.show()

sns.scatterplot(x='speechiness', y='duration_ms', hue='target', data=data_spotify_sin_outliers, )
plt.title("Gráfico de Dispersión de Nivel de Lírica")
plt.xlabel("Índice de letra en la canción")
plt.ylabel("Duración de la canción en ms")
plt.show()

sns.scatterplot(x='acousticness', y='duration_ms', hue='target', data=data_spotify_sin_outliers, )
plt.title("Gráfico de Dispersión de Acústica")
plt.xlabel("Índice de acusticidad")
plt.ylabel("Duración de la canción en ms")
plt.show()

sns.scatterplot(x='instrumentalness', y='duration_ms', hue='target', data=data_spotify_sin_outliers, )
plt.title("Gráfico de Dispersión de Instrumentalidad")
plt.xlabel("Índice de instrumentalidad")
plt.ylabel("Duración de la canción en ms")
plt.show()

sns.scatterplot(x='liveness', y='duration_ms', hue='target', data=data_spotify_sin_outliers, )
plt.title("Gráfico de Dispersión de Canciones en vivo")
plt.xlabel("Confiabilidad de canción en vivo")
plt.ylabel("Duración de la canción en ms")
plt.show()

sns.scatterplot(x='valence', y='duration_ms', hue='target', data=data_spotify_sin_outliers, )
plt.title("Gráfico de Dispersión de Valencia")
plt.xlabel("Nivel de valencia")
plt.ylabel("Duración de la canción en ms")
plt.show()

sns.scatterplot(x='tempo', y='duration_ms', hue='target', data=data_spotify_sin_outliers, )
plt.title("Gráfico de Dispersión de Tempo")
plt.xlabel("Tempo de la canción")
plt.ylabel("Duración de la canción en ms")
plt.show()

sns.scatterplot(x='time_signature', y='duration_ms', hue='target', data=data_spotify_sin_outliers, )
plt.title("Gráfico de Dispersión de Firma de tiempo")
plt.xlabel("Firma de tiempo de la canción")
plt.ylabel("Duración de la canción en ms")
plt.show()

sns.scatterplot(x='chorus_hit', y='duration_ms', hue='target', data=data_spotify_sin_outliers, )
plt.title("Gráfico de Dispersión de Entrada al coro")
plt.xlabel("Tiempo al entrar al coro")
plt.ylabel("Duración de la canción en ms")
plt.show()

sns.scatterplot(x='sections', y='duration_ms', hue='target', data=data_spotify_sin_outliers, )
plt.title("Gráfico de Dispersión de Secciones")
plt.xlabel("Cantidad de secciones")
plt.ylabel("Duración de la canción en ms")
plt.show()

In [ ]:
# Contar la cantidad de datos con Outcome 1 y 0
outcome_counts = data_spotify_sin_outliers['target'].value_counts()

# Crear el histograma
plt.bar(outcome_counts.index, outcome_counts.values, color=['blue', 'green'])
plt.xlabel('Clase')
plt.ylabel('Cantidad de datos')
plt.title('Cantidad de datos por tipo de Canción')
plt.xticks([0, 1], ['Flop', 'Hit'])
plt.show()